In [ ]:
import torch
import torch.nn as nn
import matplotlib
matplotlib.use('Agg')#('Qt5Agg')
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import savemat
import math
import time
#from models.fishnet2 import fishdw2, fishdw, fish150
#net = fishdw(num_classes=2, norm="bn", act="lrelu")
from models.fishnet2_2d import fishdeep1_2d, fishdeep10_2d
import torch.nn.functional as F

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob


# device = torch.device("cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
torch_device = torch.device("cuda")
num_workers = 0

#dirs = sorted(os.listdir("outs"))
#list(enumerate(dirs))

In [ ]:

ckpoint_path = "/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/epoch[00371]_tr[0.867]_va[0.861]_te[0.915]_trW[0.952]_vaW[0.928]_teW[0.917].pth.tar"
dir_save, fname_save = os.path.split(ckpoint_path)  #"/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/"

In [ ]:
num_classes = 10
net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)
state = torch.load(ckpoint_path)["network"]

first_key = next(iter(state))
if isinstance(net, nn.DataParallel):
    if first_key.startswith("module."):
        net.load_state_dict(state)
    else:
        new_dict = {"module." + key: value for key, value in state.items()}
        net.load_state_dict(new_dict)
else:
    if first_key.startswith("module."):
        new_dict = {key.replace("module.", ""): value for key, value in state.items()}
        net.load_state_dict(new_dict)
    else:
        net.load_state_dict(state)

#noet = net.module
net.to(torch_device)
net.eval()
print("Net Load")

len(new_dict.keys())
len(net.state_dict().keys())

In [ ]:
from visualizes.GradCam2d import GradCam2d
#from visualizes.GradCam import GradCam
from visualizes.GuidedBackpropReLUModel import GuidedBackpropReLUModel

mode_cam = "gdcam"
#"gdcam"

from datas.TomoLoader import TomoLoader,find_classes
from datas.preprocess3d import TRAIN_AUGS_3D, TEST_AUGS_3D, crop_shape, size_z
from datas.preprocess25d import TRAIN_AUGS_25D, TEST_AUGS_25D, TRAIN_AUGS_25D_v4, TEST_AUGS_25D_v4
from datas.preprocess2d import TRAIN_AUGS_2D, TEST_AUGS_2D

from datas.TomoLoader import TomoLoader
from datas import TomoPatchLoader

pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
mode_class = 0
reset_class = False
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'
classes_, class_to_idx = find_classes(data_path, pats_class, mode_class)

In [ ]:
names_part = ["tail_layer"]
idxs_layer = [2]

In [ ]:
# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
# mode_class = 0
# reset_class = False
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

# pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
mode_class = 0
reset_class = True
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'


tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    

In [ ]:
pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
mode_class = 0
reset_class = False
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

# pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'




tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    

In [ ]:
# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
# mode_class = 0
# reset_class = False
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
mode_class = 0
reset_class = True
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'




tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    

In [ ]:
# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
# mode_class = 0
# reset_class = False
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
mode_class = 0
reset_class = True
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'val'

# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'




tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/val/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/val/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/val/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    

In [ ]:
# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
# mode_class = 0
# reset_class = False
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'test'

pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
mode_class = 0
reset_class = True
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'train'

# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'




tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/train/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/train/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/train/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    

In [ ]:
pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
mode_class = 0
reset_class = False
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'val'

# pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'train'

# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'




tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/val/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/val/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/val/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    
    

In [ ]:
pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
mode_class = 0
reset_class = False
data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'train'

# pats_exclude = ('230427',)+('230713',)#+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('00_RA_12h',) + ('00_RA_24h',) + ('05_ctl',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/' + 'train'

# pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)
# pats_class = ('HD09_',)+('HD11_',)+('PD02_',)+('PD12_',)+('HD18_',)+('BJ13_',)+('PD19_',)+('PD20_',)#+('BJ01_',)+('BJ04_',)+('BJ07_',)+('BJ11_',)+('BJ12_',)+('BJ14_',)+('BJ17_',)+('BJ18_',)+('BJ21_',)+('BJ22_',)+('BJ23_',)+('HD02_',)+('HD03_',)+('HD04_',)+('HD05_',)+('HD07_',)+('HD12_',)+('HD14_',)+('HD15_',)+('HD25_',)
# mode_class = 0
# reset_class = True
# data_path = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/' + 'test'




tform_test = TEST_AUGS_25D_v4
tform_train = TRAIN_AUGS_25D_v4

for name_part,idx_layer in zip(names_part,idxs_layer):
    

    loader = TomoLoader(data_path, 1,
        transform=tform_test, aug_rate=0,
        num_workers=num_workers, shuffle=False, drop_last=False,
        pats_exclude = pats_exclude,pats_class = pats_class,
        reset_class = reset_class, mode_class = mode_class)
    print(len(loader.dataset.imgs))
    
    save_path = dir_save + '/'+ mode_cam + '/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/'+ mode_cam + '_png/'   + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'
    

    idxs_del = []
    for idx_dset, (path, cls) in enumerate(loader.dataset.imgs):
        count_er= 0
        start_index = 0
        indices_slash = []
        for i in range(len(path)):
            j = path.find("/",start_index)
            if(j!=-1):
                start_index = j+1
                indices_slash.append(j+1)
                count_er+=1

        file_name = path[indices_slash[-3]:-3] + ".mat"
        #print(save_path+file_name)

        if os.path.isfile(save_path+file_name):
            idxs_del.append(idx_dset)
    print(idxs_del)
    loader.dataset.imgs = list(filter(lambda x: loader.dataset.imgs.index(x) not in idxs_del, loader.dataset.imgs))
    #loader.dataset.imgs.pop(idxs_del)
    print(len(loader.dataset.imgs))



    del net
    net = fishdeep10_2d(input_c=12, num_classes=num_classes, norm="in", act="lrelu")


    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    state = torch.load(ckpoint_path)["network"]

    first_key = next(iter(state))
    if isinstance(net, nn.DataParallel):
        if first_key.startswith("module."):
            net.load_state_dict(state)
        else:
            new_dict = {"module." + key: value for key, value in state.items()}
            net.load_state_dict(new_dict)
    else:
        if first_key.startswith("module."):
            new_dict = {key.replace("module.", ""): value for key, value in state.items()}
            net.load_state_dict(new_dict)
        else:
            net.load_state_dict(state)
    print("Net Load")
    del state,new_dict

    #noet = net.module
    net.to(torch_device)
    net.eval()
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    #try:
    for i, (input_, target_, inputDir_) in enumerate(loader):

        
        file_name = inputDir_[0][inputDir_[0].find("/train/") + 1:-3] + ".mat"
        if os.path.isfile(save_path + file_name):
            continue


        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        with torch.no_grad():
            output_, *_ = net(input_)
            
        gdcam0, _ = grad_cam(input_, 0)
        gdcam0 = gdcam0#.detach().cpu()
        gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],num_classes))

        img = input_[0].detach().cpu().numpy()
        fig = plt.figure(figsize=(20, 20))
        ax = plt.subplot(1, 3, 1)
        ax.set_title("RI")
        plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        output_ = output_[0].detach().cpu()
        _, idx_pred = output_.max(dim=0)


        for idx_class in range(num_classes):
            torch.cuda.empty_cache()
            gdcam_, idx_pred_ = grad_cam(input_, idx_class, mode_cam)
            gdcam_stack[:,:,idx_class] = gdcam_

            if idx_class == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[target_[0]]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            
            if idx_class == idx_pred_:
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_pred_]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del input_, output_, idx_pred_

        file_name = inputDir_[0][inputDir_[0].find("/train/") + 1:-3] + ".png"
        os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
        os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
        plt.show()
        plt.savefig(img_path + file_name, dpi=fig.dpi)

        file_name = inputDir_[0][inputDir_[0].find("/train/") + 1:-3] + ".mat"
        dict_data = {'gdcam_stack':gdcam_stack}#,
        #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
        savemat(save_path+file_name, dict_data)

        plt.close()
        del dict_data, fig, gdcam_stack
        
        torch.cuda.empty_cache()

    # except:
    #     print('exception')
    del grad_cam
    torch.cuda.empty_cache()
    